In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

from sklearn.datasets import load_iris


In [10]:
# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [9]:
# Iris データセットの読み込み
x, t = load_iris(return_X_y=True)

x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [11]:
#DataSetに格納
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [12]:
# datasetの分割
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

n_train, n_val, n_test

(90, 30, 30)

In [13]:
# ランダムに分割する
torch.manual_seed(0)

train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [14]:
len(train), len(val), len(test)

(90, 30, 30)

In [15]:
# ミニバッチ学習
batch_size = 10

# Dataloaderに格納、trainだけshuffle
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [16]:
# モデル定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.L1 = nn.Linear(4, 4)
        self.L2 = nn.Linear(4, 3)

    def forward(self, x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        return x

In [79]:
# 目的関数の設定
criterion = nn.CrossEntropyLoss()

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [80]:
# 学習モデルループ実装

max_epoch = 3

# インスタンス化
torch.manual_seed(0)
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

# 学習ループ
for epoch in range(max_epoch):
    for batch in train_loader:
        x, t = batch
        
        x = x.to(device)
        t = t.to(device)
        
        optimizer.zero_grad()
        
        y = net(x)
        
        #正解率を出力
        #print(y)
        y_label = torch.argmax(y, dim=1)
        #print(y_label)
        #print(t)      

        acc = torch.sum(y_label == t) *1.0 #int => float
        acc = acc / len(t)
        
        loss = criterion(y, t)
        print("accuracy: ", acc, "loss: ", loss.item()) # item(): tensot.Tensor => float
        loss.backward()
        optimizer.step()



accuracy:  tensor(0.5000, device='cuda:0') loss:  1.1118199825286865
accuracy:  tensor(0.5000, device='cuda:0') loss:  1.0638254880905151
accuracy:  tensor(0.3000, device='cuda:0') loss:  1.0752707719802856
accuracy:  tensor(0.9000, device='cuda:0') loss:  1.0017006397247314
accuracy:  tensor(0.1000, device='cuda:0') loss:  1.0963518619537354
accuracy:  tensor(0.6000, device='cuda:0') loss:  0.9922469258308411
accuracy:  tensor(0.9000, device='cuda:0') loss:  0.9230359792709351
accuracy:  tensor(0.5000, device='cuda:0') loss:  0.8453548550605774
accuracy:  tensor(0.5000, device='cuda:0') loss:  0.8222911953926086
accuracy:  tensor(0.2000, device='cuda:0') loss:  0.9794113039970398
accuracy:  tensor(0.9000, device='cuda:0') loss:  0.8902710676193237
accuracy:  tensor(0.7000, device='cuda:0') loss:  0.7849392890930176
accuracy:  tensor(0.3000, device='cuda:0') loss:  0.8998504877090454
accuracy:  tensor(0.7000, device='cuda:0') loss:  0.8202922940254211
accuracy:  tensor(0.5000, device='

In [81]:
def calc_acc(data_loader):
    with torch.no_grad(): # 勾配計算しない
        accs = []
        
        for batch in data_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) *1.0 #int => float
            acc = acc / len(t)
            print(acc)
            accs.append(acc) #配列に要素を追加
            
        avg_acc = torch.tensor(accs).mean()
        print('Accuracy: {:.1f}%'.format(avg_acc * 100))
            


In [82]:
calc_acc(val_loader)

tensor(0.9000, device='cuda:0')
tensor(0.6000, device='cuda:0')
tensor(0.7000, device='cuda:0')
Accuracy: 73.3%


In [83]:
calc_acc(test_loader)

tensor(0.7000, device='cuda:0')
tensor(0.6000, device='cuda:0')
tensor(0.6000, device='cuda:0')
Accuracy: 63.3%
